In [ ]:
pip install pandas

In [ ]:
import re
import pandas as pd
import json

Extracting "id", "text", "score", "explanation" and "assistance functions" from the LLM-annotation of the MultiWOZ dialogues with regular expressions

In [ ]:
with open(".txt", "r", encoding="utf-8") as file:
    text = file.read()

In [ ]:
json_pattern = r'\{(?:[^{}]|"(?:\\.|[^"])*")*\}'

In [ ]:
matches = re.findall(json_pattern, text)

In [ ]:
print(f"Dialogues recognised: {len(matches)}")

In [ ]:
data = []

In [ ]:
for match in matches:
    
    id_match = re.search(r'"id"\s*:\s*"([^"]+)"', match)
    dialog_id = id_match.group(1) if id_match else None

    
    text_match = re.search(r'"text"\s*:\s*\[(.*?)\]', match, re.DOTALL)
    text_list = re.findall(r'"(.*?)"', text_match.group(1)) if text_match else []

    
    score_match = re.search(r'"score"\s*:\s*(\d+)', match)
    score = int(score_match.group(1)) if score_match else None

    
    explanation_match = re.search(r'"explanation of score"\s*:\s*"([^"]+)"', match)
    explanation = explanation_match.group(1) if explanation_match else None

    
    assistance_match = re.search(r'"assistance functions"\s*:\s*\[(.*?)\]', match, re.DOTALL)
    assistance_list = re.findall(r'"(.*?)"', assistance_match.group(1)) if assistance_match else []

    
    data.append({
        "id": dialog_id,
        "text": text_list,
        "score": score,
        "explanation": explanation,
        "assistance_functions": assistance_list
    })

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv("multiwoz_coherence_annotation_llama3-8b.csv", index=False)

Extracting "id", "text" and "coherence" from the LLM-annotation of the annotated acts of speeches based on the MultiWOZ dialogues

In [ ]:
with open(".txt", "r", encoding="utf-8") as file:
    data = file.read()

In [ ]:
matches = re.findall(r'\{.*?\}', data, re.DOTALL)

In [ ]:
print(len(matches))

In [ ]:
results = []

In [ ]:
for match in matches:
    
    id_match = re.search(r'"([\w\d]+\.json)"', match)
    dialog_id = id_match.group(1) if id_match else None

    
    text_match = re.search(r'"text"\s*:\s*\[(.*?)\]', match, re.DOTALL)
    text_list = re.findall(r'"(.*?)"', text_match.group(1)) if text_match else []

    
    coherence_match = re.search(r'"coherence"\s*:\s*(\d+)', match)
    coherence = int(coherence_match.group(1)) if coherence_match else None

    
    results.append({
        "id": dialog_id,
        "text": text_list,
        "coherence": coherence
    })

In [ ]:
df_acts = pd.DataFrame(results)

In [ ]:
df_acs = df_acts.dropna(subset=["id", "text", "coherence"])

In [ ]:
df_acs = df_acts.reset_index(drop=True)

In [ ]:
df_acts.to_csv("acts_of_speeches_annotation_llama3-8b.csv", index=False)